# Working with PreparedTask

This notebook shows how to define your own `PreparedTask`, run it with `BatchResponses`, and apply it to pandas data with `.ai.task`.


## 1. Setup

Make sure `OPENAI_API_KEY` (or Azure equivalents) is configured before running API calls.


In [1]:
import os

from openai import OpenAI
from pydantic import BaseModel, Field

from openaivec import BatchResponses, PreparedTask, pandas_ext

assert os.getenv("OPENAI_API_KEY") or os.getenv("AZURE_OPENAI_BASE_URL"), (
    "Set OPENAI_API_KEY or Azure OpenAI environment variables before running this notebook."
)


## 2. Define a custom `PreparedTask`

A `PreparedTask[T]` combines instructions and structured response schema (`T`).


In [2]:
class FruitProfile(BaseModel):
    fruit: str = Field(description="Fruit name from input")
    color: str = Field(description="Typical fruit color")
    taste: str = Field(description="Short taste description")

fruit_profile_task = PreparedTask[FruitProfile](
    instructions=(
        "Return a compact fruit profile with name, typical color, and taste. "
        "Keep answers short and factual."
    ),
    response_format=FruitProfile,
)

fruit_profile_task


PreparedTask(instructions='Return a compact fruit profile with name, typical color, and taste. Keep answers short and factual.', response_format=<class '__main__.FruitProfile'>)

## 3. Run with `BatchResponses.of_task`

API options (for example `temperature`) are passed at call site.


In [3]:
client = OpenAI()

batch = BatchResponses.of_task(
    client=client,
    model_name="gpt-4.1-mini",
    task=fruit_profile_task,
    temperature=0.0,
)

batch.parse(["apple", "banana", "grape"])


Processing batches:   0%|          | 0/3 [00:00<?, ?item/s]

[FruitProfile(fruit='apple', color='red, green, or yellow', taste='sweet to tart'),
 FruitProfile(fruit='banana', color='yellow', taste='sweet and creamy'),
 FruitProfile(fruit='grape', color='green, red, or purple', taste='sweet to slightly tart')]

## 4. Apply the same task to pandas


In [4]:
import pandas as pd

pandas_ext.set_client(client)
pandas_ext.set_responses_model("gpt-4.1-mini")

fruits = pd.Series(["apple", "orange", "strawberry"], name="fruit")
profiles = fruits.ai.task(fruit_profile_task, temperature=0.0)
profiles


Processing batches:   0%|          | 0/3 [00:00<?, ?item/s]

0    fruit='apple' color='red, green, or yellow' ta...
1    fruit='orange' color='orange' taste='citrusy a...
2    fruit='strawberry' color='red' taste='sweet an...
Name: fruit, dtype: object

## 5. Discover prebuilt tasks from the registry


In [5]:
from openaivec.task import get_task, list_tasks

list_tasks("nlp")[:6]

sentiment_task = get_task("nlp.sentiment_analysis")
sentiment_task


PreparedTask(instructions='Analyze sentiment and emotions in the input text.\n\nReturn sentiment class, sentiment confidence, emotions, emotion scores, polarity, and subjectivity.\n\nProvide natural-language fields in the same language as the input text. Do not switch explanation language unless the task explicitly requires translation.\n\nIMPORTANT: Keep predefined categorical values in exact English tokens. The following fields must use English values only: sentiment, emotions.', response_format=<class 'openaivec.task.nlp.sentiment_analysis.SentimentAnalysis'>)